# Reverse Engineering the UR3e IK parameters

Universal Robots posts the [DH parameters](https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/) of all of their robots online, the hope is with these parameters for our robot, and the joint position read off the modbus, we might be able to reverse-engineering the inverse kinematics and write our own controller for working with a floating coordinate system.

In [2]:
import numpy as np
import json
import os

In [28]:
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])

print("Available files:")
files = []
for file in os.listdir():
    if file.endswith(".json"):
        print("\t" + file)
        files.append(file)

i = 3

path = os.path.abspath("..\\IK_Scans") + "\\" + files[i]

print(path)

with open(path, 'r') as infile:
    json_data = json.load(infile)


print(start_joints)

print(joints_at_points[1])

Available files:
	Test_08_07__17_50.json
	Test_08_07__18_03.json
	Test_08_07__18_09.json
	Test_08_07__18_16.json
	Test_08_07__18_22.json
	Test_08_07__18_26.json
c:\Users\ander\OneDrive - UW\Robotics lab material\Robotics Control Code\Ben's Control Code\IK_Scans\Test_08_07__18_16.json
[[3.092], [5.599], [1.639], [2.363], [5.538], [1.323]]
[[[15, 15, -15], [-15, 15, 30]], [[3.001], [5.345], [1.641], [3.239], [5.818], [1.308]]]


In [51]:
#               theta,a,d,alpha     (radians and meters)
DH = np.array([[0, 0, 0.1519,np.pi/2],
                    [0,-0.2434,0.,0.],
                    [0,-0.2132,0.,0.],
                    [0,0.,0.1311,np.pi/2],
                    [0,0.,0.0854,-np.pi/2],
                    [0,0.,0.0921,0.]])


# print(DH)

start_joints = json_data["Start joints"]
joints_at_points = json_data["joints_at_points"]

start_joints = np.array(start_joints).flatten()

DH[:,0] = start_joints

print(DH)

# print(DH[0,:])

joints=[]

#=-(SIN(O6))*COS(R6)

# O = DH[:,0], P = DH[:,1], Q = DH[:,2], R = DH[:,3]

for i in range(6):
    joints.append(np.array([[np.cos(DH[i,0]), -np.sin(DH[i,0]) * np.cos(DH[i,3]), np.sin(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.cos(DH[i,0])],
                            [np.sin(DH[i,0]), np.cos(DH[i,0]) * np.cos(DH[i,3]), -np.cos(DH[i,0]) * np.sin(DH[i,3]), DH[i,1] * np.sin(DH[i,0])],
                            [0., np.sin(DH[i,3]), np.cos(DH[i,3]),DH[i,2]],
                            [0.,0.,0.,1.]]))

(T1,T2,T3,T4,T5,T6) = tuple(joints)

final_result = T1 * T2 * T3 * T4 * T5 * T6



[[ 3.092       0.          0.1519      1.57079633]
 [ 5.599      -0.2434      0.          0.        ]
 [ 1.639      -0.2132      0.          0.        ]
 [ 2.363       0.          0.1311      1.57079633]
 [ 5.538       0.          0.0854     -1.57079633]
 [ 1.323       0.          0.0921      0.        ]]
[[-6.76906081e-03  3.31323850e-51 -0.00000000e+00 -0.00000000e+00]
 [ 1.44314554e-02 -1.55407146e-51  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -0.00000000e+00  2.29584502e-49  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
